In [ ]:
!pip uninstall -q numpy tensorflow-estimator tensorflow folium imgaug -y
!pip install -q "albumentations==0.5.2" "numpy<1.19.0" "tensorflow-estimator<2.4.0,>=2.3.0" "tensorflow<2.4"
!pip install -q git+https://github.com/mjkvaak/ImageDataAugmentor

     |████████████████████████████████| 72 kB 497 kB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 459 kB 24.7 MB/s 
     |████████████████████████████████| 320.6 MB 32 kB/s 
     |████████████████████████████████| 948 kB 46.1 MB/s 
     |████████████████████████████████| 37.1 MB 49 kB/s 
     |████████████████████████████████| 2.9 MB 15.6 MB/s 
     |████████████████████████████████| 49.9 MB 16 kB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D,
)
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam, Adamax

In [ ]:
class_count = 78
height = 224
width = 224
channels = 3
img_shape = (height, width, channels)
IMG_SIZE = (height, width)


def create_model(class_count):
    base_model = tf.keras.applications.EfficientNetB1(
        include_top=False, weights="imagenet", input_shape=img_shape, pooling="max"
    )
    x = base_model.output
    x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
    x = Dense(
        256,
        kernel_regularizer=regularizers.l2(l=0.016),
        activity_regularizer=regularizers.l1(0.006),
        bias_regularizer=regularizers.l1(0.006),
        activation="relu",
    )(x)
    x = Dropout(rate=0.45, seed=123)(x)
    output = Dense(class_count, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(
        Adamax(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model


import json


def load_lookup_map():
    return {
        int(k): v for k, v in json.load(open("drive/MyDrive/bird-classes.json")).items()
    }


lookup_map = load_lookup_map()

In [ ]:
model = create_model(class_count)
model.load_weights("drive/MyDrive/birds-birds-95.87.h5")

27033600/27018416 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
from PIL import Image
import numpy as np


def preprocess_photo(filename):
    img = Image.open(open(filename, "rb"))
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(IMG_SIZE)
    return np.expand_dims(np.asarray(img), axis=0)

In [ ]:
predict_species(random.choice(photos))

'Best prediction: Streptopelia orientalis (94.7%)'

In [ ]:
def predict_species(photo):
    pred = model.predict(preprocess_photo(photo))
    best_pred = pred.argmax()
    best_score = round(pred[0][best_pred] * 100, 1)
    species = lookup_map[best_pred]
    return f"Best prediction: {species} ({best_score}%)"